In [3]:
import requests 

In [4]:
#Using BeautifulSoup to parse HTML link  
from bs4 import BeautifulSoup 

In [5]:
#Using pandas to make match table into a dataframe 
#To ensure no future warning with the passing of raw HTMl strings, wrapping content with StringIO
import pandas as pd
from io import StringIO

In [6]:
#Scraping multiple season data
season_years = [2024,2023,2022]

In [7]:
all_regular_games = []

In [8]:
import time 
for year in season_years:
    standings_url = f"https://www.pro-football-reference.com/years/{year}/index.htm"
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text, "html.parser")

    #Extracting team links
    standings_table = soup.select("table a")
    links = [l.get("href") for l in standings_table]
    links = [l for l in links if '/teams/' in l]
    team_urls = [f"https://www.pro-football-reference.com{l}" for l in links]

    previous_season = soup.select_one("a.prev").get("href")
    standings_url = f"https://www.pro-football-reference.com{previous_season}"

    for each_team in team_urls:
        #extracting team abbreviation
        team_name = each_team.split("/")[-2]

        
        #Getting the page of current team
        team_page = requests.get(each_team)
        games_table = pd.read_html(StringIO(team_page.text), match="Schedule & Game Results")[0]

        
        team_soup = BeautifulSoup(team_page.text)
        links = [l.get("href") for l in team_soup.find_all('a')]


        games_table.columns = games_table.columns.droplevel(0)
        games_table["Season"] = year
        games_table["Team"] = team_name
        
        regular_games = games_table["Week"].astype(str).str.isnumeric()
        games = games_table[regular_games]

        all_regular_games.append(games)

        time.sleep(5)  

In [9]:
match_df = pd.concat(all_regular_games, ignore_index = True)

In [10]:
match_df.columns = [c.lower() for c in match_df.columns]

In [11]:
match_df.to_csv("Regular_season_data.csv")